In [1]:
# importing libraries
import joblib
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf

from models.logstic_regression import Logistic_Regression
from prototype.XGBoost import XGBoost
from prototype.naive_bayes import Naive_Bayes
from prototype.rnn import RNN
from prototype.cnn import CNN
from prototype.bert import BERT
from prototype.bilstm import BiLSTM

# Load the TextPreprocessor class (assumed to be defined already)
from textpreprocessor import TextPreprocessor

import warnings
warnings.filterwarnings("ignore")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2024-10-16 00:24:41.734464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 00:24:41.742985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 00:24:41.745438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 00:24:41.752369: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 00:24:42.132220: W tensorflow/compiler/tf2

In [2]:
NUM_SAMPLE = 10000
TEST_RATIO=0.2
BATCH_SIZE=32
EPOCHS = 5
MAX_WORD_COUNT = 5000
MAX_LENGTH = 100
OUTPUT_RESULT_DIR = "Output/result"
OUTPUT_MODELS_DIR = "Output/models"
USE_TEST_DATA = True


os.makedirs(OUTPUT_RESULT_DIR, exist_ok=True)
os.makedirs(OUTPUT_MODELS_DIR, exist_ok=True)

In [3]:
# Initialize the Text Preprocessor
processor = TextPreprocessor(MAX_WORD_COUNT, MAX_LENGTH)

if USE_TEST_DATA:
    INPUT_DIR = f"Output/proto_models_rev2_{NUM_SAMPLE}"
    # Load data
    df_train = pd.read_csv(os.path.join(INPUT_DIR, 'train_cleaned.csv'))
    df_test = pd.read_csv(os.path.join(INPUT_DIR, 'test_cleaned.csv'))
    # df_test = processor.load_data()
    X_train = df_train['review']
    X_test = df_test['review']
    y_train = df_train['polarity']
    y_test = df_test['polarity']
    # X_train_seq_padded = pickle.load(os.path.join(INPUT_DIR, 'X_train_pad.pkl'))
    # X_test_seq_padded = pickle.load(os.path.join(INPUT_DIR, 'X_test_pad.pkl'))
else:
    # Load data
    df_train, df_test = processor.parallel_load_data()

    df_train_step1 = processor.remove_stopwords(df_train.copy())
    df_test_step1 = processor.remove_stopwords(df_test.copy())

    print('----------TRAIN DATA----------')
    df_train_step2 = processor.filter_by_length_of_sentence(df_train_step1.copy(),50)
    print('----------TEST DATA----------')
    df_test_step2 = processor.filter_by_length_of_sentence(df_test_step1.copy(),50)

    df_train_step3 = processor.sampling_data(df_train_step2, NUM_SAMPLE)
    df_test_step3 = processor.sampling_data(df_test_step2, int(NUM_SAMPLE*TEST_RATIO))

    # Preprocess data
    df_train_step3 = processor.map_polarity(df_train_step3.copy())
    df_test_step3 = processor.map_polarity(df_test_step3.copy())

    # Split data
    X_train, y_train = processor.split_data(df_train_step3)
    X_test, y_test = processor.split_data(df_test_step3)

X_train_tfidf, X_test_tfidf = processor.vectorize_text(X_train, X_test)
X_train_pad, X_test_pad = processor.tokenization_and_padding(X_train, X_test)

In [4]:
# Dictionary to store the results
results = {
    'Model': [],
    'Training-Time':[],
    'Accuracy': [],
    'Precision (Class 0)': [],
    'Precision (Class 1)': [],
    'Recall (Class 0)': [],
    'Recall (Class 1)': [],
    'F1-Score (Class 0)': [],
    'F1-Score (Class 1)': []
}

# Function to calculate accuracy and classification report
def evaluate_model(training_time, model_name, y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred) * 100
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Store the results
    results['Model'].append(model_name)
    results['Training-Time'].append(training_time)
    results['Accuracy'].append(accuracy)
    results['Precision (Class 0)'].append(report['0']['precision'])
    results['Precision (Class 1)'].append(report['1']['precision'])
    results['Recall (Class 0)'].append(report['0']['recall'])
    results['Recall (Class 1)'].append(report['1']['recall'])
    results['F1-Score (Class 0)'].append(report['0']['f1-score'])
    results['F1-Score (Class 1)'].append(report['1']['f1-score'])

01. Logistic Regression

In [5]:
logistic_regression = Logistic_Regression()
logistic_regression.train_model(X_train_tfidf, y_train)
y_pred_lg = logistic_regression.model.predict(X_test_tfidf)

> 01. Random SearchCV

In [6]:
logistic_regression.random_search(X_tfidf=X_train_tfidf, y=y_train, n_iter=7200, cv=3, verbos=0, random_state=42, n_jobs=-1)

Fitting 3 folds for each of 3600 candidates, totalling 10800 fits


/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/

> 02. Grid SearchCV

In [7]:
logistic_regression.grid_search(X_tfidf=X_train_tfidf, y=y_train, n_iter=7200, cv=2, verbos=0, random_state=42, n_jobs=-1,best_params=logistic_regression.random_search_cv.best_params_)

/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/woong/AI/

In [8]:
logistic_regression.train_best_model(X_train_tfidf, y_train, logistic_regression.grid_search_cv.best_params_)

In [9]:

y_pred_lg = logistic_regression.model.predict(X_test_tfidf)
y_pred_lg_random = logistic_regression.random_search_cv.best_estimator_.predict(X_test_tfidf)
y_pred_lg_grid = logistic_regression.grid_search_cv.best_estimator_.predict(X_test_tfidf)
y_pred_lg_best = logistic_regression.best_model.predict(X_test_tfidf)

# Call the function with your actual predictions (replace placeholders with your data)
evaluate_model(logistic_regression.training_time, logistic_regression.model_name, y_test, y_pred_lg)
evaluate_model(logistic_regression.random_search_time,  logistic_regression.model_name + '_random_search', y_test, y_pred_lg_random)
evaluate_model(logistic_regression.grid_search_time,  logistic_regression.model_name + '_grid_search', y_test, y_pred_lg_grid)
evaluate_model(logistic_regression.best_training_time,  logistic_regression.model_name + '_best', y_test, y_pred_lg_best)

df_results = pd.DataFrame(results)
print(df_results)

                               Model  Training-Time  Accuracy  \
0                Logistic_Regression       0.019191     83.90   
1  Logistic_Regression_random_search     209.059363     83.75   
2    Logistic_Regression_grid_search      25.479595     83.20   
3           Logistic_Regression_best       0.063323     83.15   

   Precision (Class 0)  Precision (Class 1)  Recall (Class 0)  \
0             0.820426             0.853922          0.818589   
1             0.821995             0.849732          0.811870   
2             0.818286             0.842667          0.801792   
3             0.817352             0.842527          0.801792   

   Recall (Class 1)  F1-Score (Class 0)  F1-Score (Class 1)  
0          0.855465            0.819507            0.854693  
1          0.858175            0.816901            0.853933  
2          0.856369            0.809955            0.849462  
3          0.855465            0.809497            0.848947  


02. XGBoost

In [10]:
xgboost = XGBoost()
xgboost.train_model(X_train_tfidf, y_train)

> 01. Random SearchCV

In [11]:
xgboost.random_search(X_train_tfidf, y_train, n_iter=2, cv=2, verbos=0, random_state=42, n_jobs=-1)

Best parameters found: {'tree_method': 'hist', 'subsample': 0.75, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.125, 'gamma': 0.0, 'device': 'cuda', 'colsample_bytree': 0.9}


In [12]:
xgboost.grid_search(X_train_tfidf, y_train, cv=2, verbos=0, n_jobs=-1, best_params=xgboost.random_search_cv.best_params_)

Best parameters found by GridSearchCV: {'colsample_bytree': 0.9500000000000001, 'device': 'cuda', 'gamma': 0.05, 'learning_rate': 0.135, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 0.775, 'tree_method': 'hist'}


In [13]:
xgboost.train_best_model(X_train_tfidf, y_train, xgboost.grid_search_cv.best_params_)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9500000000000001, device='cuda',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.05,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.135, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [16]:
y_pred_prob = xgboost.model.predict(xgboost.convert_to_dmatrix(X_test_tfidf, y_test))
y_pred_xgb = [1 if prob > 0.5 else 0 for prob in y_pred_prob]
y_pred_prob = xgboost.random_search_cv.best_estimator_.predict(X_test_tfidf.toarray())
y_pred_xgb_random = [1 if prob > 0.5 else 0 for prob in y_pred_prob]
y_pred_prob = xgboost.grid_search_cv.best_estimator_.predict(X_test_tfidf.toarray())
y_pred_xgb_grid = [1 if prob > 0.5 else 0 for prob in y_pred_prob]
y_pred_prob = xgboost.best_model.predict(X_test_tfidf.toarray())
y_pred_xgb_best = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Call the function with your actual predictions (replace placeholders with your data)
evaluate_model(xgboost.training_time, xgboost.model_name, y_test, y_pred_xgb)
evaluate_model(xgboost.random_search_time,  xgboost.model_name + '_random_search', y_test, y_pred_xgb_random)
evaluate_model(xgboost.grid_search_time,  xgboost.model_name + '_grid_search', y_test, y_pred_xgb_grid)
evaluate_model(xgboost.best_training_time,  xgboost.model_name + '_best', y_test, y_pred_xgb_best)

df_results = pd.DataFrame(results)
print(df_results)

df_results.to_excel(os.path.join(OUTPUT_RESULT_DIR,'Model_Compare.xlsx'))

                                Model  Training-Time  Accuracy  \
0                 Logistic_Regression       0.019191     83.90   
1   Logistic_Regression_random_search     209.059363     83.75   
2     Logistic_Regression_grid_search      25.479595     83.20   
3            Logistic_Regression_best       0.063323     83.15   
4                             XGBoost       0.781533     78.80   
5               XGBoost_random_search       4.546022     73.65   
6                 XGBoost_grid_search      75.894948     74.20   
7                        XGBoost_best       0.236399     45.25   
8                             XGBoost       0.781533     78.80   
9               XGBoost_random_search       4.546022     73.65   
10                XGBoost_grid_search      75.894948     74.20   
11                       XGBoost_best       0.236399     45.25   

    Precision (Class 0)  Precision (Class 1)  Recall (Class 0)  \
0              0.820426             0.853922          0.818589   
1        

In [36]:
# Make predictions
y_pred_prob = xgboost.random_search_cv.best_estimator_.predict(X_test_tfidf.toarray())
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 73.65%
Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.78      0.73       893
           1       0.80      0.70      0.75      1107

    accuracy                           0.74      2000
   macro avg       0.74      0.74      0.74      2000
weighted avg       0.74      0.74      0.74      2000



In [37]:
# Make predictions
y_pred_prob = xgboost.grid_search_cv.best_estimator_.predict(X_test_tfidf.toarray())
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 74.20%
Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.79      0.73       893
           1       0.80      0.71      0.75      1107

    accuracy                           0.74      2000
   macro avg       0.74      0.75      0.74      2000
weighted avg       0.75      0.74      0.74      2000



03. Naive Bayes

In [6]:
naive_bayes = Naive_Bayes()
nb_model = naive_bayes.train_model(X_train_tfidf, y_train)

# Making predictions
y_pred_nb = nb_model.predict(X_test_tfidf)

04. Recurrent Neural Network

In [7]:
rnn = RNN(processor.max_features, processor.max_length, EPOCHS, BATCH_SIZE)
rnn_model = rnn.train_model(X_train_pad, y_train, X_test_pad, y_test)

# Evaluate the model
y_pred_prob = rnn_model.predict(X_test_pad)
y_pred_rnn = (y_pred_prob > 0.5).astype("int32")

Epoch 1/5


I0000 00:00:1727101668.249242  274736 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727101668.249338  274736 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727101668.249359  274736 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727101668.257179  274736 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727101668.257281  274736 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-23

  7/313 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.5324 - loss: 0.6915

I0000 00:00:1727101669.308641  275004 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.5833 - loss: 0.6512 - val_accuracy: 0.7875 - val_loss: 0.4625
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.8693 - loss: 0.3161 - val_accuracy: 0.7625 - val_loss: 0.5223
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.9598 - loss: 0.1249 - val_accuracy: 0.7790 - val_loss: 0.5855
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.9878 - loss: 0.0452 - val_accuracy: 0.7205 - val_loss: 0.8109
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.9901 - loss: 0.0299 - val_accuracy: 0.7310 - val_loss: 0.9411
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


05. Convolutional Neural Network

In [8]:
cnn = CNN(processor.max_features, processor.max_length, EPOCHS, BATCH_SIZE)
cnn_model = cnn.train_model(X_train_pad, y_train, X_test_pad, y_test)

# Evaluate the model
y_pred_prob = rnn_model.predict(X_test_pad)
y_pred_cnn = (y_pred_prob > 0.5).astype("int32")

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6564 - loss: 0.6014 - val_accuracy: 0.8415 - val_loss: 0.3569
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9142 - loss: 0.2205 - val_accuracy: 0.8420 - val_loss: 0.3617
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9886 - loss: 0.0561 - val_accuracy: 0.8505 - val_loss: 0.4702
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9999 - loss: 0.0082 - val_accuracy: 0.8510 - val_loss: 0.5384
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.8500 - val_loss: 0.5776
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


06. Bidirectional Encoder Representations from Transformers(BERT)

In [9]:
bert = BERT(processor.max_features, processor.max_length, EPOCHS, BATCH_SIZE)
bert_model = bert.train_model(X_train, y_train, X_test, y_test)

# Predict on test data
y_pred_prob = bert_model.predict({'input_ids': bert.X_test_tokens['input_ids'], 'attention_mask': bert.X_test_tokens['attention_mask']}).logits
y_pred_bert = np.argmax(y_pred_prob, axis=-1)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
313/313 [==============================] - 88s 255ms/step - loss: 0.3044 - accuracy: 0.8696 - val_loss: 0.2210 - val_accuracy: 0.9105
Epoch 2/5
313/313 [==============================] - 79s 252ms/step - loss: 0.1445 - accuracy: 0.9510 - v

07. Bidirectional Long Short-Term Memory (BiLSTM)

In [10]:
bilstm = BiLSTM(processor.tokenizer, EPOCHS, BATCH_SIZE)
bilstm_model = bilstm.train_model(X_train_pad, y_train, X_test_pad,y_test)

# Evaluate the model
y_pred_prob = bilstm_model.predict(X_test_pad)
y_pred_bilstm = (y_pred_prob > 0.5).astype("int32")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.6264 - loss: 0.6304 - val_accuracy: 0.8070 - val_loss: 0.4262
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8621 - loss: 0.3271 - val_accuracy: 0.8330 - val_loss: 0.3909
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9140 - loss: 0.2187 - val_accuracy: 0.8295 - val_loss: 0.4609
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9451 - loss: 0.1458 - val_accuracy: 0.8180 - val_loss: 0.5280
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9718 - loss: 0.0886 - val_accuracy: 0.8045 - val_loss: 0.6203
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [14]:
# Dictionary to store the results
results = {
    'Model': [],
    'Training-Time':[],
    'Accuracy': [],
    'Precision (Class 0)': [],
    'Precision (Class 1)': [],
    'Recall (Class 0)': [],
    'Recall (Class 1)': [],
    'F1-Score (Class 0)': [],
    'F1-Score (Class 1)': []
}

# Function to calculate accuracy and classification report
def evaluate_model(model_class, y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred) * 100
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Store the results
    results['Model'].append(model_class.model_name)
    results['Training-Time'].append(model_class.training_time)
    results['Accuracy'].append(accuracy)
    results['Precision (Class 0)'].append(report['0']['precision'])
    results['Precision (Class 1)'].append(report['1']['precision'])
    results['Recall (Class 0)'].append(report['0']['recall'])
    results['Recall (Class 1)'].append(report['1']['recall'])
    results['F1-Score (Class 0)'].append(report['0']['f1-score'])
    results['F1-Score (Class 1)'].append(report['1']['f1-score'])

# Call the function with your actual predictions (replace placeholders with your data)
evaluate_model(logistic_regression, y_test, y_pred_logistic)
evaluate_model(xgboost, y_test, y_pred_xgb)
evaluate_model(naive_bayes, y_test, y_pred_nb)
evaluate_model(rnn, y_test, y_pred_rnn)
evaluate_model(cnn, y_test, y_pred_cnn)
evaluate_model(bert, y_test, y_pred_bert)
evaluate_model(bilstm, y_test, y_pred_bilstm)

# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Create the directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save result to an Excel file
df_results.to_excel(os.path.join(OUTPUT_DIR, f'result(epoch{EPOCHS}_batch{BATCH_SIZE}_sample{NUM_SAMPLE}_ratio{TEST_RATIO}).xlsx'), index=False)

In [16]:
df_results

,Model,Training-Time,Accuracy,Precision (Class 0),Precision (Class 1),Recall (Class 0),Recall (Class 1),F1-Score (Class 0),F1-Score (Class 1)
0,Logistic_Regression,0.025306,84.35,0.837233,0.849558,0.843238,0.843750,0.840225,0.846644
1,XGBoost,1.120255,82.40,0.823651,0.824324,0.813525,0.833984,0.818557,0.829126
2,Naive_Bayes,0.002810,81.95,0.808425,0.830508,0.825820,0.813477,0.817030,0.821904
3,RNN,44.071815,73.10,0.750000,0.716192,0.673156,0.786133,0.709503,0.749534
4,CNN,4.505993,73.10,0.750000,0.716192,0.673156,0.786133,0.709503,0.749534
5,BERT,402.121258,92.05,0.904055,0.937310,0.936475,0.905273,0.919980,0.921013
6,BiLSTM,30.718415,80.45,0.803738,0.805207,0.793033,0.815430,0.798350,0.810286


In [15]:

# Save Models
joblib.dump(logistic_model, os.path.join(OUTPUT_DIR, 'logistic_regression_model.pkl'))
joblib.dump(xgb_model, os.path.join(OUTPUT_DIR, 'xgboost_model.pkl'))
joblib.dump(nb_model, os.path.join(OUTPUT_DIR, 'naive_bayes_model.pkl'))
rnn_model.save(os.path.join(OUTPUT_DIR, 'rnn_model.h5'))  # or .tf
cnn_model.save(os.path.join(OUTPUT_DIR, 'cnn_model.h5'))
bert_model.save(os.path.join(OUTPUT_DIR, 'bert_model'), save_format='tf')
bilstm_model.save(os.path.join(OUTPUT_DIR, 'bilstm_model.h5'))

INFO:tensorflow:Assets written to: Output/proto_models/bert_model/assets


INFO:tensorflow:Assets written to: Output/proto_models/bert_model/assets


In [17]:
joblib.dump(processor.tokenizer, os.path.join(OUTPUT_DIR, 'processor_tokenizer.pkl'))
joblib.dump(bert.X_train_tokens, os.path.join(OUTPUT_DIR, 'bert_X_train_tokens.pkl'))
joblib.dump(bert.X_test_tokens, os.path.join(OUTPUT_DIR, 'bert_X_test_tokens.pkl'))

['Output/proto_models/bert_X_test_tokens.pkl']